In [ ]:
import pandas as pd


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
rpi = pd.read_csv("../2023/rpi.csv", index_col=False)

In [ ]:
rpi.columns

In [ ]:
rpi_cols = [
    "RPI",
    "Record",
    "Team"
]

In [ ]:
rpi = rpi.loc[:, rpi_cols]

In [ ]:
rpi.head()

In [ ]:
sos = pd.read_csv("../2023/sos.csv", index_col=False)

In [ ]:
sos.head()